In [1]:
import numpy as np

In [82]:
import keras.layers
import tensorflow as tf
import keras.backend as K
import keras.optimizers

In [35]:
def loss_function_NN(data:tf.Tensor, header:list, classification:tf.Tensor) -> tf.Tensor:
    '''
    The function to calculate and tell the model how bad it performs prediction.

    Args: 
        data: a tensor(tensorflow will automatically convert numpy array to tensorflow array),
        recording the data of a batch. 1st index represents a data point, 2nd index represents
        the values of each column at that grid point

        header: a list storing the name of variables telling how variable temperature_gradient,
        velocity_magnitude_gradient, z_velocity_gradient are correlated the 2nd index of the data.

        classification: a 1D tensor, storing classification results between 0-1 for each grid points for this batch.

    Returns: 
        loss: a SCALAR(single-value) tensor representing how bad a model predicts in a batch. A point with
        high gradient and low classification value, or low gradient and high classification value will contribute
        to higher loss. The loss wil also be high if the classificaition is close to 0.5, to encourage certain classification results.
    '''
    # Extract the indices of the gradients from the header
    temp_grad_idx = header.index('temperature_gradient')
    vel_mag_grad_idx = header.index('velocity_magnitude_gradient')
    z_vel_grad_idx = header.index('z_velocity_gradient')

    # Extract the gradient values from the data tensor
    temperature_gradient = data[:, temp_grad_idx]
    velocity_magnitude_gradient = data[:, vel_mag_grad_idx]
    z_velocity_gradient = data[:, z_vel_grad_idx]

    # Calculate the primary gradient loss
    gradient_sum = temperature_gradient + velocity_magnitude_gradient + z_velocity_gradient
    loss_high_class_low_grad = classification * (1 - gradient_sum)
    loss_low_class_high_grad = (1 - classification) * gradient_sum

    primary_loss = tf.reduce_mean(loss_high_class_low_grad + loss_low_class_high_grad)
    
    # Add regularization loss to encourage certain properties in classification
    regularization_loss = tf.reduce_mean(tf.square(classification - 0.5))
    
    # Total loss
    loss = primary_loss + regularization_loss

    return loss

In [57]:
class CustomMasking(keras.layers.Layer):
    '''
    Add a mask(Boolean mark) for points with NaN values.
    '''
    def __init__(self, mask_value=np.nan, **kwargs):
        super(CustomMasking, self).__init__(**kwargs)
        self.mask_value = mask_value

    def build(self, input_shape):
        super(CustomMasking, self).build(input_shape)

    def call(self, inputs):
        mask = K.not_equal(inputs, self.mask_value)
        return tf.convert_to_tensor(K.switch(mask, inputs, K.constant(np.nan, dtype=inputs.dtype)))

    def compute_output_shape(self, input_shape):
        return input_shape

In [31]:
class NaNHandlingLayer(keras.layers.Layer):
    '''
    Let points with NaN values have NaN classification result.
    '''
    def call(self, inputs):
        non_nan_mask = tf.math.logical_not(tf.math.is_nan(inputs))
        outputs = tf.convert_to_tensor(tf.where(non_nan_mask, inputs, tf.constant(np.nan, dtype=inputs.dtype)))
        return outputs

In [21]:
header = ['temperature_gradient','velocity_magnitude_gradient','z_velocity_gradient']

In [46]:
input = np.array([
    [0.9, 0.5, 0.4],
    [np.nan, np.nan, np.nan],
    [0.1, 0.4, 0.5],
    [0.9, 0.8, 0.5],
])

In [85]:
def NN_model(header: list) -> keras.models.Model:
    '''
    Function to create a Neural Network model.

    Args:
        header: A list of headers representing the parameters.

    Returns:
        Model: A Keras Model object.
    '''
    
    input_layer = keras.Input(shape=(len(header),))
    masked = CustomMasking(mask_value=np.nan)(input_layer)# Exclude NaN values. They will be kept as NaN.
    handled = NaNHandlingLayer()(masked)
    x = keras.layers.Dense(len(header), activation='relu')(handled) # relu is a function: when x<0, y=0; when x>0, y=x.
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dense(len(header), activation='relu')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dense(len(header), activation='relu')(x)
    output = keras.layers.Dense(1, activation='sigmoid')(x) # An s-curve whose output is between 0-1.
    
    model=tf.keras.Model(inputs=input_layer, outputs=output)
    model.compile(optimizer=tf.keras.optimizers.Adam(), loss=loss_function_NN)

    return model

model = NN_model(header)

In [ ]:
class LossHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

In [ ]:
train, val = sklearn.model_selection.train_test_split(data, test_size=test_size, random_state=42) # Use a 42(or any other number) as a seed so the result of split is always the same 
loss_hist = LossHistory()
model.fit(train,None,batch_size=batch_size,epochs=epochs,validation_data = (val,None), callbacks=[loss_hist])

In [ ]:
    plt.figure(figsize=(10, 6))
    plt.plot(loss_hist.losses, label='Loss over batches')
    plt.xlabel('Batch number')
    plt.ylabel('Loss')
    plt.title('Loss Over Batches')
    plt.legend()
    plt.plt.savefig(path)